In [ ]:
import numpy
from catboost import CatBoostRegressor, cv, Pool
import polars as pl
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gc
import numpy as np
import joblib as jl
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    MinMaxScaler,
    OrdinalEncoder,
    LabelBinarizer,
    OneHotEncoder,
    StandardScaler,
)
from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.model_selection import train_test_split
import random

SEED = 108
random.seed(SEED)
train_path = "/home/manpm/Developers/kaggle/data/mushrooms/train.csv"
test_path = "/home/manpm/Developers/kaggle/data/mushrooms/test.csv"

In [ ]:
# Prepare data
train = pd.read_csv(train_path)
print(f"train size: {train.shape}")
X_test = pd.read_csv(test_path)
print(f"test size: {X_test.shape}")
submit_df = pd.DataFrame()
submit_df["id"] = X_test["id"]


# prepare columns
target = "class"

X_train = train.drop(columns=[target], axis=1)
y_train = train[target]

categorical_cols = (
    train.drop(columns=target, axis=1).select_dtypes(include="object").columns.to_list()
)


numerical_cols = train.select_dtypes(include="number").columns.to_list()

# get top 10 most frequent names
n = 15
for c in categorical_cols:
    train_mode_values = X_train[c].value_counts()[:n].index.tolist()
    X_train.loc[~X_train[c].isin(train_mode_values), c] = "other"
    test_mode_values = X_test[c].value_counts()[:n].index.tolist()
    X_test.loc[~X_test[c].isin(test_mode_values), c] = "other"
    gc.collect()

for c in categorical_cols:
    X_train[c] = pd.Series(X_train[c], dtype="category")
    X_test[c] = pd.Series(X_test[c], dtype="category")

In [3]:
train_pool = Pool(
    X_train,
    label=y_train,
    cat_features=categorical_cols,
)

params = {
    "task_type": "GPU",  # Use GPU for training
    "verbose": 0,
    "loss_function": "Logloss",
    "learning_rate": 0.0696294726051571,
    "depth": 10,  # Reasonable depth value
    "min_data_in_leaf": 1,  # Equivalent to min_child_weight
    "bagging_temperature": 0.9405269471473167,  # Equivalent to subsample
    "l2_leaf_reg": 0.9746051811186938,  # L2 regularization
}


scores = cv(
    pool=train_pool,
    params=params,
    fold_count=5,
    plot="True",
    num_boost_round=1000,
    stratified=True,
    metric_period=50,
    early_stopping_rounds=50,
    return_models=True,
)